In [16]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import sympy as sp

In [17]:
# Define the function of the dynamics
def cosine_dynamics(X, t, constant, alpha, sigma, B):
    x,y = X # x = angle = theta, y = dxdt = angular velocity = omega, dydt = angular acceleration

    conditions = [(x > (B - sigma)) & (x < (B + sigma)),
              (x <= (B - sigma)) | (x >= (B + sigma))]

    functions = [lambda x: 1 + np.cos(2 * np.pi * (x - B) / (2 * sigma)), lambda x: 0]

    dxdt = y
    dydt = constant * (x - np.sign(B) * alpha * np.piecewise(x, conditions, functions)) # minuses the cosine
    return [dxdt, dydt]

In [34]:
#function to get max velocity for a specific alpha
def get_max_velocity(alpha, cos_start, cos_acc0, constant):
    # get sigma and B
    sigma = sp.Symbol('sigma')
    # X = cos_acc0
    B = sigma + cos_start
    equation = constant * (cos_acc0 - alpha * (1 + sp.cos((2 * sp.pi * (cos_acc0 - B)) / (2 * sigma)))) 
    solutions = sp.solve(equation, sigma)
    try: 
        sigma = float(solutions[1]) ## stability width
    except TypeError:
        return None,None,None
    B = sigma + cos_start
    # print(B, sigma)
    max_velocity = None
    for i in np.arange(cos_start, cos_acc0, 0.0001):
        # print(i)
        X = [i,0]
        t = np.linspace(0, 5, 100000)
        sol = odeint(cosine_dynamics, X, t, args=(constant, alpha, sigma, B)) #integrates dxdt and dydt to get theta (angle) and omega (velocity)
        theta = np.rad2deg(sol[:,0])
        omega = np.rad2deg(sol[:,1])
        # print(theta)
        # print(np.rad2deg(cos_acc0))
        if (any(x < 0 for x in omega)):
            max_velocity = max(omega)
            break
    if isinstance(max_velocity, (int, float)):
        return max_velocity, B, sigma
    else: return None,None,None

In [43]:
def get_parameters(mass, height, setpoint_baseline, sd_baseline, maxV):
    #Constants
    scaling = 0.971
    L = 1.09
    g = 9.81
    m = mass #change
    H = height #change
    I = 0.35*m*H**2     # Distributed Inertia
    b = 0
    maxAngle = 65
    constant = scaling * m * g * L / I

    ## baselines
    setpoint_baseline = setpoint_baseline #change
    sd_baseline = sd_baseline #change
    cos_start = np.deg2rad(setpoint_baseline + 1 * sd_baseline) 
    cos_acc0 = np.deg2rad(setpoint_baseline + 3 * sd_baseline) 
    # print(cos_start, cos_acc0, constant)

    alpha_out = 0
    velocity_out = 0
    for alpha in np.arange(0.01, 0.04, 0.0001):
        max_velocity, B, sigma = get_max_velocity(alpha, cos_start, cos_acc0, constant)
        if isinstance(max_velocity, (int, float)):
            if max_velocity < maxV:
                alpha_out = alpha
                B_out = B
                sigma_out = sigma
                velocity_out = max_velocity
            else:
                print("alpha: ", alpha_out)
                print("peak: ", np.rad2deg(B_out))
                print("width: ", np.rad2deg(sigma_out))
                print(velocity_out)
                break
        else:
            continue



## P4

In [44]:
get_parameters(69, 1.72, 0.404, 0.266, 2.011) #avg_sd + global maxV
# abrupt, like a wall, going into it, feels like a wall

alpha:  0.01809999999999995
peak:  1.6357906014216916
width:  0.9657906014216916
2.0071240029304507


In [45]:
get_parameters(69, 1.72, 0.404, 0.584, 2.011) # sd_everything + global maxV

alpha:  0.026899999999999896
peak:  2.840223034490924
width:  1.8522230344909238
2.0012591127169026


In [46]:
get_parameters(69, 1.72, 0.404, 0.266, 1.440) # avg_sd + personal maxV
## speed bump

alpha:  0.015899999999999963
peak:  1.551539410551427
width:  0.881539410551427
1.428141228638181


In [47]:
get_parameters(69, 1.72, 0.404, 0.584, 1.440) # sd_everything + personal maxV

alpha:  0.02489999999999991
peak:  2.729191334794012
width:  1.741191334794012
1.4237698079899825


## P2

In [48]:
get_parameters(51, 1.63, 0.110, 0.363, 2.011) #avg_sd + global maxV

alpha:  0.017999999999999954
peak:  1.7882588672581103
width:  1.3152588672581105
2.0064247672522217


In [49]:
get_parameters(51, 1.63, 0.110, 0.423, 2.011) # sd_everything + global maxV

alpha:  0.01969999999999994
peak:  2.0141688961269164
width:  1.481168896126916
1.9915344258543102


In [50]:
get_parameters(51, 1.63, 0.110, 0.363, 2.295) # avg_sd + personal maxV

alpha:  0.018899999999999945
peak:  1.8320527720724302
width:  1.3590527720724301
2.2679513339529285


In [51]:
get_parameters(51, 1.63, 0.110, 0.423, 2.295) # sd_everything + personal maxV

alpha:  0.020699999999999934
peak:  2.0655437459883075
width:  1.5325437459883073
2.2864209924947443


## P3

In [54]:
get_parameters(77, 1.65, 0.818, 0.262, 2.011) #avg_sd + global maxV

alpha:  0.02129999999999993
peak:  1.9507537056665025
width:  0.8707537056665025
1.981898117467969


In [55]:
get_parameters(77, 1.65, 0.818, 0.367, 2.011) # sd_everything + global maxV

alpha:  0.023899999999999914
peak:  2.347371471606372
width:  1.1623714716063718
1.9804651804147335


In [56]:
get_parameters(77, 1.65, 0.818, 0.262, 2.605) # avg_sd + personal maxV

alpha:  0.02359999999999992
peak:  2.016336801177448
width:  0.9363368011774478
2.5926854974479974


In [57]:
get_parameters(77, 1.65, 0.818, 0.367, 2.605) # sd_everything + personal maxV

alpha:  0.026199999999999904
peak:  2.429513382463942
width:  1.2445133824639416
2.5973909393024246
